In [14]:
import moderngl
import numpy as np
from PIL import Image


window_width = 2000
window_height = 2000
color = (1.0, 1.0, 0.0)
bg_color = (0.3, 0.3, 0.3)

ctx = moderngl.create_standalone_context() #create the standalone context check doc


#create the program object from ctx. it holds the vertex shader and fragment shader
#must containt #version330
prog = ctx.program(
    vertex_shader='''
        #version 330

        in vec2 in_vert;
        in vec3 in_color;

        out vec3 v_color;

        void main() {
            v_color = in_color;
            gl_Position = vec4(in_vert, 0.0, 1.0);
        }
    ''',
    fragment_shader='''
        #version 330

        in vec3 v_color;

        out vec3 f_color;

        void main() {
            f_color = v_color;
        }
    ''',
)

#the below simply scales the triangle
resize_triangle_factor = 0.7

#the X coordinates of the 3 points which are (X, y)
x = resize_triangle_factor * np.array([-1, 0, 1])
#the Y coordinates of the 3 points which are (x, Y)
y = resize_triangle_factor * np.array([-1, 1, -1])

#3 arrays called rgb which contains the color of each triangle. has to be same
#length as x and y
#the color[] scales the array full of ones. vars go from 0 to 1
r = color[0] * np.ones(len(x))
g = color[1] * np.ones(len(x))
b = color[2] * np.ones(len(x))

#the ACTUAL vertexes to be buffered into the GPU have to have the rgb for each x and y
vertices = np.dstack([x, y, r, g, b])

#create vertex buffer object vbo from context ctx and its function .buffer()
#they have to be float32 which is why we use .astype('f4' which stands for 4-byte float)
#then it has to be given in bytes to the gpu so .tobytes()
vbo = ctx.buffer(vertices.astype('f4').tobytes())
#now we create the vertex array object vao from the context ctx
#and its call fx .simple_vertex_array()
#see doc for more options
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_color')

#we crete the frame buffer object fbo which opens the window
fbo = ctx.simple_framebuffer((window_width, window_height))
fbo.use()
fbo.clear(bg_color[0], bg_color[1], bg_color[2])
#we .render( as triangles ) the VERTEX array object
vao.render(moderngl.TRIANGLES)

#we take the frame buffer object and give it to PIL to given an image
Image.frombytes('RGB', fbo.size, fbo.read(), 'raw', 'RGB', 0, -1).show()

In [9]:
vertices.astype('f4')

array([[[0. , 0. , 1. , 1. , 0. ],
        [0.5, 1. , 1. , 1. , 0. ],
        [1. , 0. , 1. , 1. , 0. ]]], dtype=float32)